In [ ]:
# Celda 1
import duckdb
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Librerías importadas correctamente")

✅ Librerías importadas correctamente


In [ ]:
# Celda 2: Cargar los datos a DuckDB
con = duckdb.connect()

# 1. Cargar los logs de acceso
con.sql("""
    CREATE OR REPLACE TABLE access_logs AS 
    SELECT * FROM '../data/logs_access_logs.json'
""")

# 2. Cargar los errores y servidores
con.sql("""
    CREATE OR REPLACE TABLE error_logs AS 
    SELECT * FROM '../data/logs_access_logs.json'
""")

con.sql("""
    CREATE OR REPLACE TABLE servers AS 
    SELECT * FROM '../data/logs_access_logs.json'
""")

# 3. Verificar que las tablas se hayan creado correctamente
print("Tablas creadas:")
con.sql("SHOW TABLES").show()

📊 Tablas creadas:
┌─────────────┐
│    name     │
│   varchar   │
├─────────────┤
│ access_logs │
│ error_logs  │
│ servers     │
└─────────────┘



In [ ]:
#Query 1: Exploración Inicial

from IPython.display import display #importante para mostrar dataframes en Jupyter

# Ver las primeras 5 filas para entender las columnas
print("Muestra de access_logs:")
display(con.sql("SELECT * FROM access_logs LIMIT 5").df())


print("Cantidad de datos:")
display(con.sql("SELECT COUNT(*) AS total_rows FROM access_logs").df())

print("Periodo de los datos:")
display(con.sql("""
    SELECT FIRST_VALUE(timestamp) OVER (ORDER BY timestamp) AS first_timestamp,
            LAST_VALUE(timestamp) OVER (ORDER BY timestamp ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_timestamp
    FROM access_logs
    LIMIT 1
""").df())

print("Usuarios únicos:")
display(con.sql("""
    SELECT COUNT(DISTINCT user_id) AS unique_users
    FROM access_logs
""").df())

Muestra de access_logs:


,log_id,timestamp,service_id,server_id,trace_id,span_id,method,endpoint,status_code,response_time_ms,request_size_bytes,response_size_bytes,user_agent,client_ip,user_id
0,1,2024-03-13 05:06:46,8,7,trace-9abe2e3d3055ff05,span-71608dc3,POST,/api/auth/login,404,40,326,57577,Mozilla/5.0,96.123.163.20,<NA>
1,2,2025-07-17 16:26:44,2,12,trace-a73a9c8658488e40,span-56a0a80f,GET,/api/orders,503,22770,39474,501,PostmanRuntime/7.29.0,6.55.171.77,5545
2,3,2024-04-17 16:14:55,10,2,trace-d3a7df6aa5c8c23f,span-650546b0,POST,/api/products,200,180,15453,22187,python-requests/2.28.0,48.246.10.118,<NA>
3,4,2024-09-08 19:06:05,3,4,trace-6646f8891d89bcb5,span-4e81d949,POST,/api/users,200,40,11293,17883,PostmanRuntime/7.29.0,33.70.37.102,<NA>
4,5,2024-01-14 11:16:27,2,12,trace-5f33d7ef9443422a,span-ed32e9bf,GET,/api/auth/login,400,424,39337,31075,Mozilla/5.0,9.180.191.49,8624


Cantidad de datos:


,total_rows
0,5000


Periodo de los datos:


,first_timestamp,last_timestamp
0,2024-01-01 03:01:31,2025-12-22 23:43:23


Usuarios únicos:


,unique_users
0,2846


In [4]:
# Query 2: Endpoints mas utilizados

display(con.sql("""
    SELECT endpoint, COUNT(endpoint) AS usage_count, 
    FROM access_logs
    GROUP BY endpoint
    ORDER BY usage_count DESC
""").df())

,endpoint,usage_count
0,/api/auth/logout,482
1,/api/users,468
2,/health,465
3,/api/products,464
4,/api/payments,458
5,/api/checkout,457
6,/api/cart,454
7,/api/auth/login,445
8,/metrics,440
9,/api/orders,438


In [5]:
# Query 3: Errores por endpoint y usuarios afectados

display(con.sql("""
    SELECT 
            endpoint,
            status_code,
            COUNT(status_code) AS total_errors,
            COUNT(DISTINCT user_id) AS unique_users,
    FROM access_logs
    GROUP BY endpoint, status_code
    ORDER BY endpoint, total_errors DESC
""").df())

,endpoint,status_code,total_errors,unique_users
0,/api/auth/login,200,119,73
1,/api/auth/login,503,41,31
2,/api/auth/login,502,37,20
3,/api/auth/login,401,34,25
4,/api/auth/login,400,34,23
...,...,...,...,...
116,/metrics,503,31,16
117,/metrics,201,30,24
118,/metrics,502,29,24
119,/metrics,500,28,21


In [6]:
# Query 4: Tiempos de respuesta por endpoint

display(con.sql("""
    SELECT
            endpoint,
            AVG(response_time_ms) AS avg_response_time,
            quantile(response_time_ms, 0.95) AS p95_response_time
    FROM access_logs
    GROUP BY endpoint
    ORDER BY avg_response_time DESC
""").df())

,endpoint,avg_response_time,p95_response_time
0,/api/auth/login,4075.984270,25143
1,/api/cart,4052.398678,24439
2,/api/auth/logout,3671.794606,23820
3,/api/checkout,3556.181619,23879
4,/health,3461.451613,21846
5,/metrics,3380.493182,23269
6,/api/users,3317.025641,22921
7,/api/search,3300.489510,22234
8,/api/products,3254.993534,22868
9,/api/orders,3152.940639,21199


In [7]:
# Query 5: Análisis temporal de las solicitudes

display(con.sql("""
    SELECT
            EXTRACT(HOUR FROM timestamp) AS hour_of_day,
            COUNT(*) AS requests_count,
            AVG(response_time_ms) AS avg_response_time,
            quantile(response_time_ms, 0.95) AS p95_response_time
    FROM access_logs    
    GROUP BY hour_of_day
""").df())

,hour_of_day,requests_count,avg_response_time,p95_response_time
0,0,203,3052.669951,22535
1,1,190,4005.336842,20780
2,2,205,2923.619512,18157
3,3,227,3578.405286,23352
4,4,231,3702.021645,24655
5,5,217,2950.976959,21208
6,6,181,3722.712707,23836
7,7,190,3749.189474,25434
8,8,178,2923.606742,23596
9,9,212,4004.150943,25375


In [8]:
# Query 6: Ranking con windows functions de endpoints por metodo HTTP según tiempo de respuesta promedio

display(con.sql("""
    WITH ranked_endpoints AS (
        SELECT
                method,
                endpoint,
                AVG(response_time_ms) AS avg_response_time,
                RANK() OVER (PARTITION BY method ORDER BY avg_response_time DESC) AS response_time_rank
        FROM access_logs
        GROUP BY method, endpoint
    )
    SELECT *
    FROM ranked_endpoints
    WHERE response_time_rank <= 5
""").df())

,method,endpoint,avg_response_time,response_time_rank
0,DELETE,/health,5122.547170,1
1,DELETE,/api/auth/logout,4971.887097,2
2,DELETE,/api/users,4431.983871,3
3,DELETE,/api/auth/login,4093.758065,4
4,DELETE,/metrics,3794.230769,5
5,GET,/api/cart,4725.775610,1
6,GET,/api/auth/login,4719.848168,2
7,GET,/api/auth/logout,3853.684729,3
8,GET,/api/checkout,3838.250000,4
9,GET,/health,3708.341232,5


In [9]:
# Query 7: Cambios en el volumen de solicitudes por hora

display(con.sql("""
    WITH hourly_traffic AS (
        SELECT
                EXTRACT(HOUR FROM timestamp) AS hour_of_day,
                COUNT(*) AS requests_count,
                AVG(response_time_ms) AS avg_response_time,
                quantile(response_time_ms, 0.95) AS p95_response_time
        FROM access_logs    
        GROUP BY hour_of_day
    )
    SELECT
        hour_of_day,
        requests_count,
        avg_response_time,
        p95_response_time,
        LAG(requests_count) OVER (ORDER BY hour_of_day) AS previous_hour_requests,
        (requests_count - previous_hour_requests) AS request_change
    FROM hourly_traffic
""").df())

,hour_of_day,requests_count,avg_response_time,p95_response_time,previous_hour_requests,request_change
0,0,203,3052.669951,22535,<NA>,<NA>
1,1,190,4005.336842,20780,203,-13
2,2,205,2923.619512,18157,190,15
3,3,227,3578.405286,23352,205,22
4,4,231,3702.021645,24655,227,4
5,5,217,2950.976959,21208,231,-14
6,6,181,3722.712707,23836,217,-36
7,7,190,3749.189474,25434,181,9
8,8,178,2923.606742,23596,190,-12
9,9,212,4004.150943,25375,178,34


# Conclusiones y Recomendaciones

Basado en el análisis de los logs del servidor, se observan los siguientes puntos críticos:

### 1. Latencia Crítica en Auth y Checkout
El tiempo de respuesta P95 en endpoints críticos como `/api/auth/login` y `/api/cart` supera los **24.000ms (24 segundos)**.
* **Recomendación:** Esto indica un bloqueo grave (posiblemente timeouts de base de datos). Es urgente revisar los índices de la tabla de usuarios y carritos, ya que bloquea el ingreso de dinero.

### 2. Inestabilidad en el Servicio de Autenticación
Se detectaron fallas graves de infraestructura en `/api/auth/login`.
* **Evidencia:** Se registraron **41 errores 503** (Service Unavailable) y **37 errores 502** (Bad Gateway), afectando a más de **50 usuarios únicos** que no pudieron ingresar.
* **Recomendación:** Revisar si el servidor se está quedando sin memoria o CPU en los momentos de mayor tráfico, ya que no llega a procesar las solicitudes de ingreso.

### 3. Patrones de Tráfico Sostenidos
El tráfico es inusualmente constante durante las 24 horas, sin caídas drásticas durante la madrugada.
* **Evidencia:** La carga oscila muy poco: el máximo es a las **16:00 y 21:00 hs** (233 peticiones) y el mínimo es a las **08:00 AM** (178 peticiones). La diferencia entre el momento más ocupado y el más tranquilo es mínima.
* **Recomendación:** Al no existir una "hora valle" clara (momentos de tráfico cero), no se puede apagar el servidor para mantenimiento sin afectar usuarios. Se recomienda realizar actualizaciones cerca de las **08:00 AM**, que es el punto de menor impacto relativo.